<a href="https://colab.research.google.com/github/AriannaHeartbell/sd-llm_example/blob/main/SD_reForge_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Original source code is [here](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb) and his [DigiClau Youtube](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw) & [☕](https://www.buymeacoffee.com/digiclau)

Since updates to [lllyasviel's Forge](https://github.com/lllyasviel/stable-diffusion-webui-forge) and [DigiClau's Forge Colab notebook](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb) have stopped (or in development?), I created this temproal reForge Colab notebook by slightly modifying DigiClau's original notebook. If DigiClau's notebook is updated in the future or a removal request is received, I will delete this notebook.

[lllyasviel님의 Forge](https://github.com/lllyasviel/stable-diffusion-webui-forge)와 [DigiClau님의 Forge Colab 노트북](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb)의 업데이트가 중단되었기에(혹은 개발 중?), DigiClau님의 원래 노트북을 reForge에 맞게 약간 수정하여 임시로 코랩 노트북을 만들었습니다. 추후에 Digiclau님의 코랩 노트북이 갱신되거나 혹은 삭체 요청을 받을 경우, 이 노트북은 삭제하겠습니다.

## ### Panchovix's [reForge](https://github.com/Panchovix/stable-diffusion-webui-reForge)

## ### Updates

- July, 2024:
 - Starting this notebook

In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion WebUI Forge 시작하는 섹션</font>
output_path = 'digiclau_Forge'

#@markdown ## This Colab Notebook will be connected to Google Drive.
#@markdown #### - A folder named 'digiclau_Forge' will be created in Google Drive, and the settings files and generated images will be saved in that folder.
#@markdown #### - Unlike the original DigiClau's forge notebook, this notebook does not download models separately, except for some extensions.
#@markdown #### - After running this notebook for the first time, you can save the necessary models (Lora, checkpoints, etc.) in the appropriate folders that will be created in your Google Drive.
#@markdown #### .
#@markdown ## 이 Colab Notebook은 구글 드라이브에 연결됩니다.
#@markdown #### - Google Drive에 'digiclau_Forge'라는 폴더가 생기고, 그 안에 세팅 파일과, 생성된 이미지들이 저장됩니다.
#@markdown #### - 원본 DigiClau님의 forge 노트북과 다르게, 일부 익스텐션을 제외하면, 따로 모델들을 다운 받지 않습니다.
#@markdown #### - 필요한 모델(Lora, checkpoints 등등)들은 이 노트북을 최초로 한 번 실행 한 뒤에 구글 드라이브에 생성되는 각 폴더들에 맞게 저장하면 됩니다.
is_drive_used = True

#@markdown ### .
#@markdown ## Some Extensions
ControlNet = True #@param{type: "boolean"}
Regional_Prompter = True #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
ADetailer = True #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}
InpaintAnything = True #@param{type: "boolean"}
DynamicPrompts = False #@param{type: "boolean"}
LayerDiffuse = True #@param{type: "boolean"}

# 로그 삭제
#Clear_Log = True #@param{type:'boolean'}

if is_drive_used: # true by default
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  # Colab에 설치
  root = '/content/'
  sd_main = 'stable-diffusion-webui-reForge'
  # 구글 드라이브에 필요한 폴터 생성
#  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/output # for SVD videos
  !mkdir -p {output_path}/checkpoints
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
  !mkdir -p {output_path}/ControlNet

  # 추가적인 모델 폴더들
  !mkdir -p {output_path}/RealESRGAN
#  !mkdir -p {output_path}/LyCORIS
  !mkdir -p {output_path}/SwinIR
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path # Is it still needed?
  root = '/content/'

def print_message(msg):
  print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  print('📍📍📍 ', msg, ' 📍📍📍')
  print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  pass

def clear():
    from IPython.display import clear_output
    return clear_output()

def install_extensions():
  # extensions directory
  %cd {root}/{sd_main}/extensions

  if ControlNet:
    print_message('ControlNet 모델 다운로드중...')
#    !git clone https://github.com/Mikubill/sd-webui-controlnet.git # ControlNet extension is built-in
#    %cd {root}/{sd_main}/extensions/sd-webui-controlnet/models
    %cd {root}/{sd_main}/models/ControlNet
    # insightface
    !pip install insightface
    %cd {root}/{sd_main}/extensions
  if Regional_Prompter:
    print_message('Regional_Prompter 확장 프로그램 설치중...')
    !git clone https://github.com/hako-mikan/sd-webui-regional-prompter
  if Ultimate_SD_Upscale:
    print_message('Ultimate_SD_Upscale 확장 프로그램 설치중...')
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
  if ADetailer:
    !git clone https://github.com/Bing-su/adetailer
    print_message('ADetailer 확장 프로그램 설치중...')
  if AnimateDiff:
    print_message('AnimateDiff 확장 프로그램 설치중...')
    !git clone https://github.com/continue-revolution/sd-forge-animatediff
    %cd {root}/{sd_main}/extensions/sd-forge-animatediff/model
#    downloadModel('https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_adapter.ckpt')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors')

    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_high.pth')
    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_mid.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.5.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.75.pth')
    downloadModel('https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.safetensors')

#    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/lora/mm_sd15_v3_adapter.safetensors')
    %cd {root}/{sd_main}/extensions
  if InpaintAnything:
    !git clone https://github.com/Uminosachi/sd-webui-inpaint-anything
  if DynamicPrompts:
    print_message('Dynamic Prompts 확장 프로그램 설치중...')
    !git clone https://github.com/adieyal/sd-dynamic-prompts.git
  if LayerDiffuse:
    print_message('Layer Diffuse 확장 프로그램 설치중...')
    !git clone https://github.com/layerdiffusion/sd-forge-layerdiffuse

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        return

  #install python 3.10.6
  !apt-get update -y
  !apt-get install python3.10.6

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config-FORGE.json'): #(only once at the first run)
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config-FORGE.json -O {output_path + '/ui-config-FORGE.json'}
  if not os.path.exists(output_path + '/config-FORGE.json'): #(only once at the first run)
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config-FORGE.json -O {output_path + '/config-FORGE.json'}
  # Use config file in google drive
  if not os.path.exists(output_path + '/styles.csv'): #(only once at the first run)
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/style.csv -O {output_path + '/styles.csv'}
  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'): #(only once at the first run)
    !touch {output_path + '/params.txt'}

  !ln -s {output_path + '/ui-config-FORGE.json'} {root}/{sd_main}/ui-config.json
  !ln -s {output_path + '/config-FORGE.json'} {root}/{sd_main}/config.json
  !ln -s {output_path + '/styles.csv'} {root}/{sd_main}/styles.csv
  !ln -s {output_path + '/params.txt'} {root}/{sd_main}/params.txt
#  !ln -s {output_path}/outputs # {root}/{sd_main}/
  !ln -s {output_path}/output # {root}/{sd_main}

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
#  %cd {root}/{sd_main}
  !rm -rf embeddings # -rf r(recursive-delete subdir), f(force-no confirmation required)
  !ln -s {output_path}/embeddings

  # link all checkpoints in the checkpoints folder
  %cd {root}/{sd_main}/models/Stable-diffusion
  checkpoints_in_google_drive = glob.glob(output_path + '/checkpoints/*')
#  print_message('구글 드라이브에 들어 있는 Checkpoint 모델: %s'%checkpoints_in_google_drive)
  for f in checkpoints_in_google_drive:
    !ln -s {f}

  # link ESRGAN upscalers in the model folder
  !mkdir -p {root}/{sd_main}/models/ESRGAN
  %cd {root}/{sd_main}/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
#  print_message('구글 드라이브에 들어 있는 Upscaler 모델: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # link RealESRGAN upscalers in the model folder
  !mkdir -p {root}/{sd_main}/models/RealESRGAN
  %cd {root}/{sd_main}/models
  !rm -rf RealESRGAN
  !ln -s {output_path}/RealESRGAN

  # link SwinIR upscalers in the model folder
  !mkdir -p {root}/{sd_main}/models/SwinIR
  %cd {root}/{sd_main}/models
  !rm -rf SwinIR
  !ln -s {output_path}/SwinIR

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/{sd_main}/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/{sd_main}/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in Google Drive
  !mkdir -p {output_path}/VAE
  %cd {root}/{sd_main}/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

  # use ControlNet folder in Google Drive
#  !mkdir -p {output_path}/ControlNet # already above
  %cd {root}/{sd_main}/models
  !rm -rf ControlNet
  !ln -s {output_path}/ControlNet

  # FORGE ONLY models
  # use svd folder in Google Drive
  !mkdir -p {output_path}/svd
  %cd {root}/{sd_main}/models
  !rm -rf svd
  !ln -s {output_path}/svd

  # use svd folder in Google Drive
  !mkdir -p {output_path}/z123
  %cd {root}/{sd_main}/models
  !rm -rf z123
  !ln -s {output_path}/z123

  updatePython()
!mkdir -p {root}
%cd {root}
# os.chdir(root)
!apt-get -y install -qq aria2
# !pip install pyngrok # no ngrok will be used on Forge
!git clone https://github.com/Panchovix/stable-diffusion-webui-reForge.git
# A1111 first launch
%cd {root}/{sd_main}
!git checkout main

!COMMANDLINE_ARGS="--exit"  python launch.py

if is_drive_used: # == GDriveSaveMode.ON:
  initSaveGoogleDriveModelOnly()

install_extensions()

%cd {root}/{sd_main}
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue'
args += ' --share '

extra_arguments = '--disable-model-loading-ram-optimization --opt-sdp-attention --theme dark'
args += ' ' + extra_arguments

!python {root}/{sd_main}/launch.py {args}


Mounted at /content/drive
/content
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: 